In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.mun.ca/regoff/calendar/sectionNo=ENGI-1120"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course links for webdriver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
tables = page_soup.find("div", {"id": "content"}).findAll("table")
len(tables)

3

In [18]:
link_text_containers = []
for table in tables:
    sub_containers = table.findAll("a", {"onclick": re.compile("showcourse")})
    for sub_container in sub_containers:
        link_text_containers.append(sub_container)
        
link_text_containers

[<a class="clink" onclick="showcourse('SCI-0910', event, this);" style="color:red;">1050</a>,
 <a class="clink" onclick="showcourse('ENGI-1247', event, this);" style="color:red;">1010</a>,
 <a class="clink" onclick="showcourse('ENGI-1248', event, this);" style="color:red;">1020</a>,
 <a class="clink" onclick="showcourse('ENGI-1249', event, this);" style="color:red;">1030</a>,
 <a class="clink" onclick="showcourse('ENGI-1250', event, this);" style="color:red;">1040</a>,
 <a class="clink" onclick="showcourse('SCI-1337', event, this);" style="color:red;">1000</a>,
 <a class="clink" onclick="showcourse('SCI-1339', event, this);" style="color:red;">1001</a>,
 <a class="clink" onclick="showcourse('SCI-1356', event, this);" style="color:red;">2050</a>,
 <a class="clink" onclick="showcourse('SCI-1579', event, this);" style="color:red;">1050</a>,
 <a class="clink" onclick="showcourse('SCI-1815', event, this);" style="color:red;">1051</a>,
 <a class="clink" onclick="showcourse('ENGI-2143', event

In [19]:
len(link_text_containers) #some are duplicate links while others have the same code but for diff subjects

99

# 2. Text run - try scraping first course

In [20]:
driver.find_element_by_link_text(link_text_containers[0].text).click()

In [21]:
page_soup = soup(driver.page_source, 'lxml')
course_info = page_soup.find("div", {"id": "tip", "class": "tip"})

In [22]:
course_info.find("p", {"class": "courseNumber"}).text.strip()

'1050'

In [23]:
course_info.find("p", {"class": "courseTitle"}).text.strip()

'General Chemistry I'

In [24]:
course_info.find("div", {"class": "courseDesc"}).p.text.strip()

'builds on basic chemistry concepts from high school. Topics include gases; thermochemistry; atomic structure; periodic properties; chemical bonding including valence bond theory; hybridization and introduction to molecular orbital theory; properties of liquids and solids.'

In [25]:
driver.find_element_by_link_text("Close").click()

In [29]:
#to find the subject name:
re.split("\d+", link_text_containers[0].parent.text)[0].strip()

'Chemistry'

In [56]:
#mapping of subject names to code:
subject_codes = {
    
    "Chemistry": "CHEM",
    "ENGI": "ENGI",
    "Mathematics": "MATH",
    "Physics": "PHYS",
    "ME": "ME",
    "Human Kinetics and Recreation": "HKR",
    "Process Engineering": "PE",
    "Electrical and Computer Engineering": "ECE",
    "Civil Engineering": "CIV",
    "Recreation": "HKR"
}

### trial to see if there's a way to keep track of which selenium links have been clicked and which haven't

In [34]:
links_found = driver.find_elements_by_link_text(link_text_containers[0].text)
links_found

[<selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="4c841185-29c7-40fe-9bb9-984c3506ecca")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="b90bcf6e-7c7e-4e4a-a7d8-c875dbbb53fd")>]

In [35]:
links_clicked = []
links_clicked.append(links_found[0])
links_clicked

[<selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="4c841185-29c7-40fe-9bb9-984c3506ecca")>]

In [36]:
driver.find_element_by_link_text(link_text_containers[0].text) in links_clicked

True

In [38]:
driver.find_elements_by_link_text(link_text_containers[0].text)[1] in links_clicked

False

In [39]:
links_found = driver.find_elements_by_link_text("003W")
links_found

[<selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="c1c57615-b5fc-4699-a92a-5c4dc9277343")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="2883a42b-cc0b-440e-895b-6a291fa1e71a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3aefceb7cad805061134108c51ef423", element="a06fc4aa-b26f-4d1a-9f7b-2aeebdd754c2")>]

In [40]:
links_clicked.append(links_found[0])

In [42]:
links_found[1] in links_clicked

False

# 3. Implement Automation script to scrape all courses

In [58]:
counter = 0
course_codes = []
course_names = []
course_descs = []

links_clicked = []

for link_text_container in link_text_containers:
    
    #find all links with that text
    links_found = driver.find_elements_by_link_text(link_text_container.text)
    time.sleep(1)
    
    #check if each link has been clicked
    #for the links that haven't been clicked, click them, scrape, and add them to the list of clicked links
    for link in links_found:
        if link not in links_clicked:
            link.click()
            time.sleep(3)
            
            page_soup = soup(driver.page_source, 'lxml')
            course_info = page_soup.find("div", {"id": "tip", "class": "tip"})
            
            subject_text = re.split("\d+", link_text_container.parent.text)[0].strip()
            if len(subject_text.split()) > 4:
                subject_text = subject_text.split()[-1]
            
            subject = subject_codes[subject_text]
            course_code = subject + " " + course_info.find("p", {"class": "courseNumber"}).text.strip()
            course_name = course_info.find("p", {"class": "courseTitle"}).text.strip()
            course_desc = course_info.find("div", {"class": "courseDesc"}).p.text.strip()
            
            course_codes.append(course_code)
            course_names.append(course_name)
            course_descs.append(course_desc)
            
            print("Scraped ", course_codes[-1])
            counter += 1
            driver.find_element_by_link_text("Close").click()
            time.sleep(3)
            links_clicked.append(link)
            break
            
print("scraped {} courses".format(counter))   

Scraped  CHEM 1050
Scraped  ENGI 1010
Scraped  ENGI 1020
Scraped  ENGI 1030
Scraped  ENGI 1040
Scraped  MATH 1000
Scraped  MATH 1001
Scraped  MATH 2050
Scraped  PHYS 1050
Scraped  PHYS 1051
Scraped  ENGI 200W
Scraped  ENGI 200W
Scraped  ENGI 3101
Scraped  ENGI 3424
Scraped  ME 3101
Scraped  ME 3102
Scraped  ME 3301
Scraped  ME 3401
Scraped  ENGI 001W
Scraped  ENGI 002W
Scraped  ENGI 4430
Scraped  ME 4302
Scraped  ME 4402
Scraped  ME 4501
Scraped  ME 4601
Scraped  ENGI 001W
Scraped  ENGI 002W
Scraped  ENGI 003W
Scraped  ENGI 4421
Scraped  ME 5103
Scraped  ME 5201
Scraped  ME 5502
Scraped  ME 5602
Scraped  ENGI 002W
Scraped  ENGI 003W
Scraped  ENGI 004W
Scraped  ME 6202
Scraped  ME 6303
Scraped  ME 6403
Scraped  ME 6701
Scraped  ME 6702
Scraped  HKR 2311
Scraped  ENGI 003W
Scraped  ENGI 004W
Scraped  ENGI 005W
Scraped  ME 7203
Scraped  ME 7704
Scraped  HKR 2311
Scraped  ENGI 004W
Scraped  ENGI 005W
Scraped  ENGI 006W
Scraped  ENGI 8152
Scraped  ME 8705
Scraped  HKR 2311
Scraped  HKR 4703

# 4. Inspect, clean, and write to CSV

In [59]:
course_codes

['CHEM 1050',
 'ENGI 1010',
 'ENGI 1020',
 'ENGI 1030',
 'ENGI 1040',
 'MATH 1000',
 'MATH 1001',
 'MATH 2050',
 'PHYS 1050',
 'PHYS 1051',
 'ENGI 200W',
 'ENGI 200W',
 'ENGI 3101',
 'ENGI 3424',
 'ME 3101',
 'ME 3102',
 'ME 3301',
 'ME 3401',
 'ENGI 001W',
 'ENGI 002W',
 'ENGI 4430',
 'ME 4302',
 'ME 4402',
 'ME 4501',
 'ME 4601',
 'ENGI 001W',
 'ENGI 002W',
 'ENGI 003W',
 'ENGI 4421',
 'ME 5103',
 'ME 5201',
 'ME 5502',
 'ME 5602',
 'ENGI 002W',
 'ENGI 003W',
 'ENGI 004W',
 'ME 6202',
 'ME 6303',
 'ME 6403',
 'ME 6701',
 'ME 6702',
 'HKR 2311',
 'ENGI 003W',
 'ENGI 004W',
 'ENGI 005W',
 'ME 7203',
 'ME 7704',
 'HKR 2311',
 'ENGI 004W',
 'ENGI 005W',
 'ENGI 006W',
 'ENGI 8152',
 'ME 8705',
 'HKR 2311',
 'HKR 4703',
 'ME 7104',
 'ME 7703',
 'ME 8604',
 'ME 7205',
 'ME 7703',
 'ME 8305',
 'PE 6202',
 'PE 7291',
 'PE 8290',
 'ME 7404',
 'ME 7405',
 'ME 8406',
 'ECE 7410',
 'ECE 8410',
 'ME 7204',
 'ME 7205',
 'ME 7603',
 'ME 8504',
 'ME 7105',
 'ME 7603',
 'ME 8106',
 'ME 8304',
 'ME 860

In [60]:
course_names

['General Chemistry I',
 'Engineering Statics',
 'Introduction to Programming',
 'Engineering Graphics and Design',
 'Mechanisms and Electric Circuits',
 'Calculus I',
 'Calculus II',
 'Linear Algebra I',
 'General Physics I: Mechanics',
 'General Physics II: Oscillations, Waves, Electromagnetism',
 'Work Term Preparation and Professional Development',
 'Work Term Preparation and Professional Development',
 'Engineering Professionalism I',
 'Engineering Mathematics',
 'Chemistry and Physics of Engineering Materials I',
 'Production Technology',
 'Dynamics',
 'Thermodynamics I',
 'Engineering Work Term 1',
 'Engineering Work Term 2',
 'Advanced Calculus for Engineering',
 'Mechanisms and Machines',
 'Thermodynamics II',
 'Fluid Mechanics I',
 'Mechanics of Solids I',
 'Engineering Work Term 1',
 'Engineering Work Term 2',
 'Engineering Work Term 3',
 'Probability and Statistics',
 'Chemistry and Physics of Engineering Materials II',
 'Mechatronics I',
 'Fluid Mechanics II',
 'Mechanics 

In [61]:
course_descs

['builds on basic chemistry concepts from high school. Topics include gases; thermochemistry; atomic structure; periodic properties; chemical bonding including valence bond theory; hybridization and introduction to molecular orbital theory; properties of liquids and solids.',
 'is the first course in Engineering mechanics. Forces and moments are described with vector algebra, leading to a description of the equilibrium conditions for particles and solid bodies. The importance of free body diagrams is highlighted. This knowledge is then applied to the analysis of trusses, frames and machines. Additional topics include an examination of friction and the concepts of centre of force, centroids and second moments of area.',
 'is an introduction to algorithmic problem solving techniques and computer programming, including basic program control structures (sequence, call, branch, loop) and data representations, functional decomposition, and design by contract. Exercises and examples are drawn

In [62]:
len(course_names) == len(course_codes) == len(course_descs)

True

In [63]:
len(course_names)

99

In [64]:
course_codes1 = []
course_names1 = []
course_descs1 = []
counter = 0

for i in range(len(course_codes)):
    if course_codes[i] not in course_codes1:
        course_codes1.append(course_codes[i])
        course_names1.append(course_names[i])
        course_descs1.append(course_descs[i])
        counter += 1
        
print(counter)

76


In [66]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes1,
    "Course Name": course_names1,
    "Course Description": course_descs1
    
})

df

,Course Number,Course Name,Course Description
0,CHEM 1050,General Chemistry I,builds on basic chemistry concepts from high s...
1,ENGI 1010,Engineering Statics,is the first course in Engineering mechanics. ...
2,ENGI 1020,Introduction to Programming,is an introduction to algorithmic problem solv...
3,ENGI 1030,Engineering Graphics and Design,provides two complementary competencies. First...
4,ENGI 1040,Mechanisms and Electric Circuits,"is offered in two serial modules, including la..."
...,...,...,...
71,PE 8276,Natural Gas Engineering,(same as the former ENGI 8676) covers process ...
72,PE 8294,Downstream Processing,(same as the former ENGI 8694) includes: oil a...
73,ME 8407,Sustainable Energy Systems,(same as the former ENGI 8984) examines thermo...
74,ME 8505,Fluid Structure Interactions,(same as the former ENGI 8964) examines struct...


In [67]:
df.to_csv('Memorial_University_MechEng_Core_and_Electives_(AllYears)_Courses.csv', index = False)

In [68]:
driver.quit()